# Getting and filtering data from production redis

In [2]:
from urllib.parse import urlparse
from dotenv import load_dotenv
import os
from pathlib import Path
import json
import redis
import pandas as pd # only if needed for filtering of data
import mysql.connector
from mysql.connector import Error

In [3]:
# load .env file
env_path = Path('../') / '.env'
load_dotenv(env_path)

True

### Getting CiC info from MySql

In [3]:
mysql_url = os.getenv('MYSQLPROD')
parsed_mysql_url = urlparse(mysql_url)

try:
    connection = mysql.connector.connect(host=parsed_mysql_url.hostname,
                                         user=parsed_mysql_url.username,
                                         password=parsed_mysql_url.password,
                                         database=parsed_mysql_url.path[1:],
                                         port=parsed_mysql_url.port)
    if connection.is_connected():
        db_Info = connection.get_server_info()
        print("Connected to MySQL Server version ", db_Info)
        cursor = connection.cursor()
        cursor.execute("SELECT * FROM cic;")
        fields = [field_md[0] for field_md in cursor.description]
        result = [dict(zip(fields,row)) for row in cursor.fetchall()]
        print(result)

except Error as e:
    print("Error while connecting to MySQL", e)
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version  8.0.28
[{'id': 'CIC-00149d9a-da31-5e61-844d-3e818b8a2ded', 'createdAt': datetime.datetime(2023, 3, 17, 1, 39, 6, 973000), 'updatedAt': datetime.datetime(2023, 6, 30, 11, 23, 12, 222000), 'addressNumber': None, 'addressStreet': None, 'availableWifiNetworks': '[]', 'isScanningForWifi': 1, 'lastScannedForWifi': datetime.datetime(2023, 6, 30, 11, 23, 12, 221000), 'updateStatus': None, 'updateUntil': None, 'wifiConnectionStatus': 'disconnected', 'wifiPassword': 'U2FsdGVkX1951NICZIs2+sxY4CiVlbEuIn4yyBLyW+o=', 'wifiSSID': 'home_devices', 'zipCode': '3452 SN', 'dayElectricityPrice': None, 'electricityPrice': None, 'gasPrice': None, 'nightElectricityPrice': None, 'silentMode': 'never', 'boilerType': 'opentherm', 'heatDeliverySystems': '["radiators", "underfloor_heating"]', 'thermostatType': 'opentherm_room_temperature', 'maximumHeatingOutdoorTemperature': 15.0, 'ratedMaximumHousePower': 7000.0, 'orderNumber': 'QUATT3293', 'status': 'active', 'lteInfo': '{}', '

In [59]:
df_sql = pd.DataFrame.from_dict(result)
df_sql_orders = df_sql[~pd.isna(df_sql['orderNumber']) & ~df_sql['orderNumber'].isin(['QUATT0000', 'QUATT00000'])][['id', 'orderNumber']]
df_sql_orders

,id,orderNumber
0,CIC-00149d9a-da31-5e61-844d-3e818b8a2ded,QUATT3293
4,CIC-008396fd-1fe4-5812-b410-91ac000e439c,QUATT72
5,CIC-0084fc13-f765-505f-ac5f-ecfd36eab409,QUATT3324
8,CIC-00ca4ee0-d7df-5bc1-b575-c90aa7ab3046,QUATT984
9,CIC-00caba21-208e-5870-bb66-127018e6e583,QUATT3224
...,...,...
2920,CIC-ff50429c-2bd8-5018-a903-afd2737e718a,QUATT4416
2921,CIC-ff67dd59-4a79-5e06-8d22-115416e4be9b,QUATT4020
2924,CIC-ff9155b8-30a7-51ea-bad2-fee0ff5eacef,QUATT271
2926,CIC-ffd3bdd6-fd98-54cc-ab62-6a89aa0e16fd,QUATT810


### Getting CiC info from Redis

In [30]:
def get_cic_stats_from_redis(redis_url):
    # Connect to Redis database
    parsed_url = urlparse(redis_url)
    r = redis.Redis(host=parsed_url.hostname,
                    port=parsed_url.port,
                    db=0,
                    password=parsed_url.password,
                    username=parsed_url.username)
    
    # get objects from redis
    redis_objects = r.mget(r.keys(pattern="cic:*CIC*lastStat*"))
    results = []
    for obj in redis_objects:
        try:
            results.append(json.loads(obj.decode()))
        except:
            pass
    return results

In [61]:
# get cic data from redis
REDIS_URL = os.getenv("REDISPROD")

redis_data = get_cic_stats_from_redis(REDIS_URL)

df_redis = pd.json_normalize(redis_data)

# select list of cic ids which are connected via Lte
cic_ids = df_redis[(df_redis['system.quattBuild'] != "2.0.1")][['system.quattId', 'system.quattBuild','time.tsHuman', 'system.isLteConnected','system.isWifiReachable','system.isEthernetReachable']]

# print list of cic ids
print(cic_ids)

                                system.quattId    system.quattBuild  \
9     CIC-4d5ecd4f-f1c7-5f02-bd6d-ae974808bd6e                1.2.6   
18                                         NaN                2.0.3   
20    CIC-0b42b575-78c3-5392-8dad-8e4837fe9f63               0.0.53   
30    CIC-0ec25c82-fd63-5871-bf1e-918246a4ca6f  01.02.06-production   
34                                         NaN                  NaN   
...                                        ...                  ...   
1873                                       NaN                  NaN   
1890  CIC-7a715ae0-c61f-5a0e-bb76-11944de2e1ef                2.0.0   
1894  CIC-853c8648-b266-5ce7-bcb0-16dea7264234                2.0.3   
1908  CIC-93ab8148-cee2-59b6-a298-c5abe8a90b1e                2.0.3   
1916  CIC-421420d6-8b26-5bc2-8f6f-e15afe281eb3              0.01.03   

                  time.tsHuman system.isLteConnected system.isWifiReachable  \
9     2023-06-27T16:43:08.821Z                  True                

In [62]:
df_redis = df_redis.set_index('system.quattId').join(df_sql_orders.set_index('id'), how='inner').reset_index()
print

<function print>

In [63]:
df_connectivity = df_redis[['system.quattId', 'system.quattBuild','time.tsHuman','system.isWifiReachable','system.isEthernetReachable', 'system.isLteReachable']].copy()
df_connectivity['isCiCInLteMode'] = [True if (row['system.isLteReachable'] and not row['system.isWifiReachable'] and not row['system.isEthernetReachable']) else False for idx, row in df_connectivity.iterrows()]
df_connectivity['isCiCInLteMode'].value_counts()

isCiCInLteMode
False    1463
True      359
Name: count, dtype: int64

In [64]:
cic_ids[['system.quattId', 'system.quattBuild','time.tsHuman', 'system.isLteConnected','system.isWifiReachable','system.isEthernetReachable']]

,system.quattId,system.quattBuild,time.tsHuman,system.isLteConnected,system.isWifiReachable,system.isEthernetReachable
9,CIC-4d5ecd4f-f1c7-5f02-bd6d-ae974808bd6e,1.2.6,2023-06-27T16:43:08.821Z,True,False,True
18,NaN,2.0.3,2023-06-30T08:51:14.336Z,True,False,False
20,CIC-0b42b575-78c3-5392-8dad-8e4837fe9f63,0.0.53,2023-07-03T13:56:41.403Z,True,False,False
30,CIC-0ec25c82-fd63-5871-bf1e-918246a4ca6f,01.02.06-production,2023-05-11T07:19:51.391Z,True,False,True
34,NaN,NaN,2023-06-07T21:34:22.323Z,True,False,False
...,...,...,...,...,...,...
1873,NaN,NaN,2023-07-03T13:56:35.439Z,True,False,False
1890,CIC-7a715ae0-c61f-5a0e-bb76-11944de2e1ef,2.0.0,2023-05-24T20:59:59.959Z,True,True,False
1894,CIC-853c8648-b266-5ce7-bcb0-16dea7264234,2.0.3,2023-07-03T13:56:24.208Z,True,True,False
1908,CIC-93ab8148-cee2-59b6-a298-c5abe8a90b1e,2.0.3,2023-07-03T13:56:33.837Z,True,True,False


### Find CiCs with conflicting controller type and redis settings

In [49]:
def map_settings_to_controller_type(row):
    ccBoilerType = row['system.ccBoilerType']
    ccThermostatType = row['system.ccThermostatType']
    ccNumberOfHeatPumps = row['system.ccNumberOfHeatPumps']
    detectedThermostatType = row['system.thermostatType']
    identifier_lookup = {'RO1': 1, 'RN1': 2, 'WO1': 3, 'WN1': 4, 'RO2': 5, 'RN2': 6, 'WO2': 7, 'WN2': 8}
    controller_select = ''
    error = False
    errorStr = ""
    
    # Check thermostat type
    if ccThermostatType == "0" or pd.isna(ccThermostatType):
        ccThermostatType = detectedThermostatType
    if ccThermostatType == "opentherm_room_temperature":
        controller_select = controller_select + "R"
    elif ccThermostatType == "opentherm_without_room_temperature":
        controller_select = controller_select + "W"
    else:
        error = True
        errorStr = errorStr + "Incorrect thermostat type,"
    
    # Check Boiler type
    if ccBoilerType == "opentherm":
        controller_select = controller_select + "O"
    elif ccBoilerType == "on_off":
        controller_select = controller_select + "N"
    else:
        error = True
        errorStr = errorStr + "Incorrect boiler type,"
    
    # Check number of heatpumps
    if ccNumberOfHeatPumps == 1:
        controller_select = controller_select + "1"
    elif ccNumberOfHeatPumps == 2:
        controller_select = controller_select + "2"
    else:
        error = True
        errorStr = errorStr + "Incorrect number of heatpumps setting"
    
    if error:
        errorStr = errorStr.strip(",")
        return errorStr
    
    return identifier_lookup[controller_select]


In [65]:
df_mismatch = df_redis[['system.quattId', 'orderNumber', 'system.ccBoilerType', 'system.ccThermostatType', 'system.ccNumberOfHeatPumps', 'qc.controllerIdentifier', 'system.thermostatType']].copy()
df_mismatch["controllerSelect"] = df_mismatch.apply(map_settings_to_controller_type, axis='columns')
df_mismatch
# df_mismatch[['system.quattId', 'qc.controllerIdentifier', 'controllerSelect']]

,system.quattId,orderNumber,system.ccBoilerType,system.ccThermostatType,system.ccNumberOfHeatPumps,qc.controllerIdentifier,system.thermostatType,controllerSelect
0,CIC-00149d9a-da31-5e61-844d-3e818b8a2ded,QUATT3293,opentherm,opentherm_room_temperature,1.0,1.0,opentherm_room_temperature,1
1,CIC-008396fd-1fe4-5812-b410-91ac000e439c,QUATT72,None,None,NaN,NaN,opentherm_room_temperature,"Incorrect boiler type,Incorrect number of heat..."
2,CIC-0084fc13-f765-505f-ac5f-ecfd36eab409,QUATT3324,on_off,opentherm_room_temperature,NaN,2.0,opentherm_room_temperature,Incorrect number of heatpumps setting
3,CIC-00ca4ee0-d7df-5bc1-b575-c90aa7ab3046,QUATT984,opentherm,opentherm_room_temperature,NaN,1.0,opentherm_room_temperature,Incorrect number of heatpumps setting
4,CIC-00caba21-208e-5870-bb66-127018e6e583,QUATT3224,opentherm,opentherm_room_temperature,NaN,1.0,opentherm_room_temperature,Incorrect number of heatpumps setting
...,...,...,...,...,...,...,...,...
1817,CIC-ff50429c-2bd8-5018-a903-afd2737e718a,QUATT4416,on_off,opentherm_room_temperature,1.0,2.0,opentherm_room_temperature,2
1818,CIC-ff67dd59-4a79-5e06-8d22-115416e4be9b,QUATT4020,on_off,opentherm_room_temperature,1.0,2.0,opentherm_room_temperature,2
1819,CIC-ff9155b8-30a7-51ea-bad2-fee0ff5eacef,QUATT271,opentherm,opentherm_room_temperature,NaN,1.0,opentherm_room_temperature,Incorrect number of heatpumps setting
1820,CIC-ffd3bdd6-fd98-54cc-ab62-6a89aa0e16fd,QUATT810,on_off,opentherm_room_temperature,NaN,2.0,opentherm_room_temperature,Incorrect number of heatpumps setting


In [67]:
df_controller_select_err = df_mismatch.loc[df_mismatch['controllerSelect'].str.startswith("Inc", na=False)]
df_controller_select = df_mismatch.loc[~df_mismatch['controllerSelect'].str.startswith("Inc", na=False)]
print("Counts of CiC's where no controller will start:")
print(df_controller_select_err["controllerSelect"].value_counts())

df_controller_select["controllerMatch"] = df_controller_select["controllerSelect"] == df_controller_select["qc.controllerIdentifier"]
print("\nCounts of CiC's where the new controller selected matches the old controller:")
print(df_controller_select["controllerMatch"].value_counts())
print("\nCiC's where a different controller will be started:")
print(df_controller_select[df_controller_select["controllerMatch"] == False][['orderNumber', 'system.quattId', "qc.controllerIdentifier", 'controllerSelect']])
# df_mismatch['controllerSelect'].str.startswith("Inc", na=False)

Counts of CiC's where no controller will start:
controllerSelect
Incorrect number of heatpumps setting                                                    1171
Incorrect boiler type,Incorrect number of heatpumps setting                                12
Incorrect thermostat type,Incorrect boiler type,Incorrect number of heatpumps setting      12
Incorrect boiler type                                                                       4
Incorrect thermostat type,Incorrect number of heatpumps setting                             2
Name: count, dtype: int64

Counts of CiC's where the new controller selected matches the old controller:
controllerMatch
True     618
False      3
Name: count, dtype: int64

CiC's where a different controller will be started:
     orderNumber                            system.quattId  \
1199   QUATT4827  CIC-a8f558fa-f92a-58b2-992b-318c1b7d38f8   
1410   QUATT4300  CIC-c4b50742-6d3e-5325-a1e1-9958be194f0a   
1418   QUATT6809  CIC-c610a4d5-f3a1-5643-ad5a-7d00fcc

C:\Users\dhruv\AppData\Local\Temp\ipykernel_23640\3595854288.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_controller_select["controllerMatch"] = df_controller_select["controllerSelect"] == df_controller_select["qc.controllerIdentifier"]
